# performance-testing-ml-serving-apis-with-locust
## https://www.analyticsvidhya.com/blog/2021/06/performance-testing-ml-serving-apis-with-locust/

## Contents:
  - Build a simple API with FastAPI
  - Build a classification model in python
  - Wrap the model with FastAPI
  - Test the API with Postman client
  - Load test with Locust

In [5]:
#!pip install fastapi
!pip install uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00


# how-to-run-fastapi-uvicorn-in-google-colab
## https://stackoverflow.com/questions/63833593/how-to-run-fastapi-uvicorn-in-google-colab

In [9]:
!pip install nest-asyncio pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 38.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19879 sha256=a577698bad675ac37029fcc62ddf0b180e28bd7cd1a8b9def3fd56638aa58e45
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


# Starting a ASGI server using FasAPI and Uvicorn

In [11]:
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from pyngrok import ngrok
from typing import Dict
from pydantic import BaseModel
import uvicorn
import numpy as np
import pickle
import pandas as pd
import json

<ipython-input-11-9b01ee4fc2eb>:3: RuntimeWarning: coroutine 'Server.serve' was never awaited
  import nest_asyncio


In [12]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)


@app.get("/")
async def root():
  return {"message": "Built with FastAPI"}

In [13]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://cdeb-35-221-18-35.ngrok.io


INFO:     Started server process [776]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     36.255.234.150:0 - "GET / HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [776]


# Building classification model 

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import joblib, pickle
import os
import yaml

# Folder to load config file
CONFIG_PATH = "."

# Function to load yaml configuration file
def load_config(config_name):
    """[The function takes the yaml config file as input and loads the config]
    Args:
        config name ([yaml]): [The function takes yaml config as input]
    Returns:
        [string]: [Returns the config]
    """
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")

In [17]:
#path to the dataset
filename = "./breast-cancer-wisconsin.csv"
#load data
data = pd.read_csv(filename)
#replace '?' with -99999
data = data.replace('?', -99999)
# drop id column
data = data.drop(config['drop_columns'], axis=1)

# Define X (independent variables) and y (target variable)
X = np.array(data.drop(config["target_name"], 1))
y = np.array(data[config['target_name']])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=config["test_size"], random_state=config["random_state"]
)

<ipython-input-17-e873385a12f4>:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(data.drop(config["target_name"], 1))


In [18]:
# call our classifier and fit to our data
classifier = KNeighborsClassifier(
    n_neighbors=config["n_neighbors"],
    weights=config["weights"],
    algorithm=config["algorithm"],
    leaf_size=config["leaf_size"],
    p=config["p"],
    metric=config["metric"],
    n_jobs=config["n_jobs"],
)

In [19]:
# training the classifier
classifier.fit(X_train, y_train)

# test our classifier
result = classifier.score(X_test, y_test)
print("Accuracy score is. {:.1f}".format(result))

# saving model to disk
pickle.dump(classifier, open('KNN_model.pkl', 'wb'))

Accuracy score is. 0.9


# Building API using FastAPI

In [20]:
# Load the model
model = pickle.load(open('KNN_model.pkl', 'rb'))

In [21]:
@app.post('/predict')
def pred(body: dict):
    """[summary]
    Args:
        body (dict): [The pred method takes Response as input which is in Json format and returns the predicted value from the saved model.]
    Returns:
        [Json]: [The pred function returns the predicted value]
    """
    # Get the data from the POST request.
    data = body
    varList = []
    for val in data.values():
      varList.append(val)
    # Make prediction from the saved model
    prediction = model.predict([varList])
    # Extract the value
    output = prediction[0]
    # return the output in the json format
    return {'The prediction is ': output}

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://fa22-35-221-18-35.ngrok.io


INFO:     Started server process [776]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     54.86.50.139:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     36.255.234.150:0

# Using PostMan to test the API

In [23]:
# postman client
# select post https://f071-35-221-18-35.ngrok.io/predict

# select Body->raw->json, paste below req dict and send a post request 
request_pred = {
  "radius_mean": 13.54,
  "texture_mean": 14.36,
  "perimeter_mean": 87.46,
  "area_mean": 566.3,
  "smoothness_mean": 0.09779,
  "compactness_mean": 0.08129,
  "concavity_mean": 0.06664,
  "concave points_mean": 0.04781,
  "symmetry_mean": 0.1885,
  "fractal_dimension_mean": 0.05766,
  "radius_se": 0.2699,
  "texture_se": 0.7886,
  "perimeter_se": 2.058,
  "area_se": 23.56,
  "smoothness_se": 0.008462,
  "compactness_se": 0.0146,
  "concavity_se": 0.02387,
  "concave points_se": 0.01315,
  "symmetry_se": 0.0198,
  "fractal_dimension_se": 0.0023,
  "radius_worst": 15.11,
  "texture_worst": 19.26,
  "perimeter_worst": 99.7,
  "area_worst": 711.2,
  "smoothness_worst": 0.144,
  "compactness_worst": 0.1773,
  "concavity_worst": 0.239,
  "concave points_worst": 0.1288,
  "symmetry_worst": 0.2977,
  "fractal_dimension_worst": 0.07259
}

# Load test using Postman

In [24]:
!pip install locust

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.1/826.1 kB 36.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.6/103.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 86.4 MB/s eta 0:00:00
  Created wheel for Flask-BasicAuth: filename=Flask_BasicAuth-0.2.0-py3-none-any.whl size=4241 sha256=9644b1656e68354fc8b55c5fc7bad009c8ccf335f677e50b43a5c240bea44dd9
  Stored in directory: /root/.cache/pip/wheels/2c/96/a5/54f6f7fa880f487e2f6c52cc9943d3299272a956cbd5af3b8c
  Created wheel for roundrobin: filename=roundrobin-0.0.4-py3-none-any.whl size=4033 sha256=3336f9a1e7ac3cd643fbda4aa7ed3063c0e9a15e0cfeced4934b

In [28]:
#run locust -f perf.py from local 